## Initialize weighting factors

In [425]:
place = [
    1900,
    1600,
    1300,
    1100,
    900,
    700,
    600,
    500,
    400,
    300,
    200,
    150,
    100,
    100,
    100,
    100,
    50,
]
place_top = len(place) - 1

internet_points = [
    500,
    420,
    340,
    290,
    235,
    185,
    160,
    130,
    105,
    80,
    55,
    40,
    25,
    25,
    25,
    25,
    15    
]
internet_top = len(internet_points) - 1

participation = {
}

params = {
    'ann_depreciation' : 0.2,
    'rep_depreciation' : 8000,
    'membership'      : 100,
    'sa_champs'       : sa_champs,
    'internet'        : internet_points,
    'inet_max_tour'   : 2,
    'part_max_tot'    : 1000,
    'part_max_opp'    : 200,
    'part'            : {'t': 40, 'c': 20, 'i': 20}    
}

## Define class player

In [467]:
from functools import reduce

class player:
    
    #initialize
    def __init__(self, name, lyom, open):
        
        #name
        self.name=name
        
        #last year of membership
        self.lyom=lyom
        
        #opening balance
        self.open=open
        
        #annual depreciation
        self.dep=int(-round(params['ann_depreciation']*self.open))
        
        #representative's depreciation
        self.rep=0
        
        #membership points
        self.mem=0
        
        #SA championship points
        self.place=0
        
        #internet championship points
        self.inet=0
        self.inet_list=[]
        
        #paticipation points
        self.part=0
        self.part_list=[]
        
        #closing balance
        self.close=0
        
    #get string representation
    def __str__(self):
        return self.name+','+self.lyom+','+str(self.open)+','\
        +str(self.dep)+','+str(self.rep)+','+str(self.mem)+','\
        +str(self.place)+','+str(self.inet)+','+str(self.part)+','+str(self.close)
    
    #compare
    def __lt__(self,other): #supposed to replace __cmp__ in python 3, doesn't seem to work
        return self.close < other.close
  
    
#     def __cmp__(self,other):
#         if self.close < other.close:
#             return -1
#         elif self.close == other.close:
#             return 0
#         else:
#             return 1
    
    #set last year of membership
    def set_lyom(self,lyom):
        self.lyom=lyom
        
    #is member?
    def is_mem(self):
        return self.lyom==data['year']
        
    #calculate membership points
    def calc_mem(self):
        self.mem=params['membership']
        
    #calculate representative's depreciation
    def calc_rep(self,s,n,p,t):
        self.rep = -int(min(round(1000*(n+2*float(s)/100)-1000*(float(t)-p)/(t-1)),params['rep_depreciation']))
        
    
    #calculate SA championship points
    def calc_place(self,pos=-1,tie=1):
        place = params['sa_champs']
        if self.is_mem():            
            if pos==-1:
                self.place=place[-1]                
            else:
                if pos+tie<=place_top:
                    shared=reduce(lambda x,y: x+y,place[pos:pos+tie])
                elif pos<place_top and top<=pos+tie:
                    shared=reduce(lambda x,y: x+y,place[pos:place_top])+(pos+tie-place_top)*place[-1]
                elif inet_top<=pos:
                    shared=tie*place[-1]
                self.place=int(round(float(shared)/tie))
                
    #calculate internet championship points
    def calc_inet(self,pos=-1,tie=1):
        inet = params['internet']
        if pos==-1 or not self.is_mem():
            self.inet_list.append(0)
        else:
            if pos==-1:
                self.inet_list.append(0)
            else:
                if pos+tie<=inet_top:
                    shared=reduce(lambda x,y: x+y,inet[pos:pos+tie])
                elif pos<inet_top and top<=pos+tie:
                    shared=reduce(lambda x,y: x+y,inet[pos:inet_top])+(pos+tie-inet_top)*inet[-1]
                elif inet_top<=pos:
                    shared=tie*inet[-1]
                self.inet_list.append(int(round(float(shared)/tie)))
                
            copy=[]
            for x in self.inet_list:
            	copy.append(x)
            copy.sort()
            copy.reverse()
            self.inet=reduce(lambda x,y: x+y,copy[0:params['inet_max_tour']])
        
    #calculate participation points
    def calc_part(self,date,opp,game_type):
        if self.is_mem():
            if game_type in ['t','c','i'] :#(type=='i' and opp.is_mem()):
                if len(self.part_list) == 0:
                    self.part_list.append([date,opp.name,game_type,\
                                           params['part'][game_type],\
                                           params['part'][game_type],\
                                           params['part'][game_type]]) #?
                else:
                    cum_tot = self.part_list[-1][-1]
                    found = 0
                    i = len(self.part_list)
                    while not found and i>0:
                        i = i-1 
                        if self.part_list[i][1] == opp.name:
                           found = 1
                    if found:
                        cum_opp = self.part_list[i][4]
                    else:
                        cum_opp = 0
                    
                    points = min(params['part_max_tot']-cum_tot,
                                 params['part_max_opp']-cum_opp,
                                 params['part'][game_type])    
                    
                    self.part_list.append([date,opp.name,game_type,points,cum_opp+points,cum_tot+points])
            
            self.part = self.part_list[-1][-1]
    
    #calculate closing balance
    def calc_close(self):
        self.close=max(self.open+self.dep+self.rep+self.mem+self.place+self.inet+self.part,0)
        
    #create report
    def report(self):
        lines=[]
        w1=40
        w2=5
        w=w1+w2
        lines.append('-'*w+'\n')
        lines.append(('WAGC points for '+self.name).ljust(w)+'\n')
        lines.append('-'*w+'\n')
        lines.append((str(data['year'])+' opening balance:').ljust(w1)+str(self.open).rjust(w2)+'\n')
        lines.append(('Annual depreciation:').ljust(w1)+str(self.dep).rjust(w2)+'\n')
        lines.append(('Representative\'s depreciation:').ljust(w1)+str(self.rep).rjust(w2)+'\n')
        lines.append(('Membership points:').ljust(w1)+str(self.mem).rjust(w2)+'\n')
        lines.append(('SA Championship points:').ljust(w1)+str(self.place).rjust(w2)+'\n')
        lines.append(('Internet Championship points:').ljust(w1)+str(self.inet).rjust(w2)+'\n')
        lines.append(('Participation points:').ljust(w1)+str(self.part).rjust(w2)+'\n')
        lines.append('-'*w+'\n')
        lines.append((str(data['year'])+' closing balance:').ljust(w1)+str(self.close).rjust(w2)+'\n')
        lines.append('-'*w+'\n')
        lines.append('\n')
        lines.append('\n')
        lines.append('\n')
                
        w1=40
        w2=5
        w=w1+w2
        lines.append('-'*w+'\n')
        lines.append('Exposition of Internet Championship points'.ljust(w)+'\n')
        lines.append('-'*w+'\n')
        for i in range(len(self.inet_list)):
            lines.append(('Tournament '+str(i+1)).ljust(w1)+str(self.inet_list[i]).rjust(w2)+'\n')
        lines.append('-'*w+'\n')
        lines.append(('Sum of '+str(params['inet_max_tour'])+' best results:').ljust(w1)+str(self.inet).rjust(w2)+'\n')
        lines.append('-'*w+'\n')
        lines.append('\n')
        lines.append('\n')
        lines.append('\n')
                
        w1=12
        w2=30
        w3=6
        w=w1+w2+w3*4
        lines.append('-'*w+'\n')
        lines.append('Exposition of participation points'.ljust(w)+'\n')
        lines.append('-'*w+'\n')
        lines.append(''.ljust(w1+w2)+'Game'.rjust(w3)+'Part'.rjust(w3)+'Cum'.rjust(w3)+'Cum'.rjust(w3)+'\n')
        lines.append('Date'.ljust(w1)+'Opponent'.ljust(w2)+'type'.rjust(w3)+'pnts'.rjust(w3)+\
                     'opp'.rjust(w3)+'tot'.rjust(w3)+'\n')
        lines.append('-'*w+'\n')
        for info in self.part_list:
            lines.append(info[0].ljust(w1)+info[1].ljust(w2)+info[2].rjust(w3)+str(info[3]).rjust(w3)+\
                         str(info[4]).rjust(w3)+str(info[5]).rjust(w3)+'\n')
        lines.append('-'*w+'\n')
        lines.append('\n')
        lines.append('\n')
        lines.append('\n')
        
        w=5
        lines.append('-'*w+'\n')
        lines.append('END'.center(w)+'\n')
        lines.append('-'*w+'\n')
        
        return lines


# Main
Input

In [489]:
import pandas as pd

data = {
    'year'     : 2014,
    'open'     : pd.read_csv('open.txt',index_col=False),
    'rep'      : pd.read_csv('rep.txt',index_col=False,\
                             names=['name','sponsorship','times','placing','tot_participants']), #according to readme.txt
    'mem'      : open('mem.txt').readlines(),
    'place'    : open('place.txt').readlines(),
    'internet' : open('inet.txt').readlines(),
    'games'    : pd.read_csv('System_Games.csv',index_col=False,\
                             names=['date','black','white','handi','komi','diff','type','result','comment'])
}
players = {}


* initialize players with opening balances
* calulate annual depreciation

In [469]:
for line in data['open'].iterrows():
    name = line[1]['Name'].strip()
    lyom = line[1]['Last year of membership']
    closing_balance = line[1][-1]    
    players[name] = player(name,lyom,closing_balance)
    print("%s:\t%s\t%s"%(name,players[name].lyom,players[name].open))



Victor Chow:	2013	10720
Andre Connell:	2013	8813
Welile Gogotshe:	2013	8579
John Leuner:	2013	8293
Samuel Scott:	2013	8262
Andrew Davies:	2013	8254
Lloyd Rubidge:	2013	6835
Chris Welsh:	2013	6434
Ben Gale:	2013	6058
David Richfield:	2013	5382
Steve Kroon:	2013	4881
Konrad Scheffler:	2012	4544
Paul Edwards:	2013	4474
Francois van Niekerk:	2013	4415
Dave Gale:	2013	4312
Bob Gale:	2013	4238
Paul Steyn:	2013	4238
Rory Shea:	2013	3632
Sipho Mampe:	2013	3068
James Nicolson:	2013	2728
Max Rabkin:	2013	2653
Aki Zhou:	2013	2560
Chris Visser:	2013	2339
Stephen Martindale:	2013	2169
Ben Bredenkamp:	2013	2094
Thabiso Lemeke:	2013	1924
Gordon Wells:	2011	1768
Tristen Taylor:	2006	1637
Marc Loon:	2013	1496
Rory Beling:	2008	1448
Reinhardt Messerschmidt:	2007	1355
James Gelant:	2012	1329
Arnold Wentzel:	2013	1317
Florian Breuer:	2009	992
Cheng Lai:	2006	934
Leander Gaum:	2006	894
Jaco Swanepoel:	2007	840
Peter Charter:	2013	801
Sakkie Buys:	2013	797
Carle Joubert:	2009	744
Andries Kruger:	2006	738
Cl

calculate representative's depreciation

In [470]:
for line in data['rep'].iterrows():    
    name = line[1]['name']    
    
    if name in players.keys():
        s = line[1]['s']
        n = line[1]['n']
        p = line[1]['p']
        t = line[1]['t']
        players[name].calc_rep(s,n,p,t)
        print ("%s: \t%s"%(name,players[name].rep))
        

John Leuner: 	-2830
Andre Connell: 	-2186


calculate membership points

In [471]:
for name in data['mem']:
    name = name.strip()
    
    if name in players.keys():
        players[name].set_lyom(data['year'])
    else:
        players[name] = player(name, data['year'], 0)
        
    players[name].calc_mem()
    
    print("%s:\t%s"%(name,players[name].mem))

Ben Bredenkamp:	100
Peet Brits:	100
Sakkie Buys:	100
Peter Charter:	100
Victor Chow:	100
Andre Connell:	100
Andrew Davies:	100
Hendrik de Ridder:	100
Antoine Dymond:	100
Paul Edwards:	100
Dylan Farre:	100
Bob Gale:	100
Ben Gale:	100
Welile Gogotshe:	100
Sally Gross:	100
Brent Harrison:	100
Clive Hunt:	100
Gerrit Kapp:	100
Kim He-Jin:	100
Steve Kroon:	100
John Leuner:	100
Marc Loon:	100
Christiaan Maasdorp:	100
Elizabeth Meyer:	100
James Nicolson:	100
Lloyd Rubidge:	100
Samuel Scott:	100
Rory Shea:	100
Allen Simpson:	100
Margot Smythe:	100
Paul Steyn:	100
Jacobus van der Merwe:	100
Francois van Niekerk:	100
Chris Visser:	100
Gordon Wells:	100
Chris Welsh:	100
Arnold Wentzel:	100
Brett Wilson:	100
Michael Yang:	100
Harvey Zhang:	100
Aki Zhou:	100


calculate SA championship points

In [472]:
pos = 0
qualified = True
qualifiers = []
others = []

for line in data['place']:
    if line.strip() == '*':
        qualified = False
    elif qualified:
        names = line.strip().split(',')        
        for name in names:
            if name in players.keys():                
                players[name].calc_place(pos,len(names))                                
                qualifiers.append(name)
                print('%s:\t%s'%(name,players[name].place))
        pos += len(names)
    elif not qualified:        
        others.append(line.strip()) 
print()
for name in (set(others) - set(qualifiers)):    
    if name in players.keys():            
        players[name].calc_place()    
        print('%s:\t%s'%(name,players[name].place))
        

Victor Chow:	1900
Andre Connell:	1600
Welile Gogotshe:	1300
Samuel Scott:	1100
John Leuner:	900
Bob Gale:	700
Andrew Davies:	600
Chris Welsh:	500
Paul Edwards:	400
Lloyd Rubidge:	300
James Nicolson:	200
Ben Bredenkamp:	150
Paul Steyn:	100
Marc Loon:	100
Peet Brits:	100
Rory Shea:	100

Ricardo Kaizer:	0
Elizabeth Meyer:	50
Christiaan Maasdorp:	50
Margot Smythe:	50
Brian Huma:	0
Brett Wilson:	50
Ricky Pilane:	0
Allen Simpson:	50
Zane Deane:	0


calculate internet championship points

In [473]:
pos = 0
participants = []

for line in data['internet']:
    if line.strip() == '*':
        for name in set(players.keys() - set(participants)):
            players[name].calc_inet()
            if players[name].inet > 0:
                print('%s:\t%s'%(name,players[name].inet))
        pos = 0
        participants = []
    else:
        names = line.split(',')
        for name in names:
            name = name.strip()
            if name in players.keys():
                players[name].calc_inet(pos,len(names))                
                if players[name].inet > 0:
                    print('%s:\t%s'%(name,players[name].inet))
                participants.append(name)
        pos += len(names)    

Andrew Davies:	500
Chris Welsh:	350
John Leuner:	350
Samuel Scott:	350
Andre Connell:	193
Paul Steyn:	193
Rory Shea:	130
Peet Brits:	105
Hendrik de Ridder:	80
Elizabeth Meyer:	55
Andrew Davies:	1000
Samuel Scott:	730
Paul Steyn:	573
John Leuner:	612
Chris Welsh:	612
Francois van Niekerk:	185
Peet Brits:	265
Christiaan Maasdorp:	130
Elizabeth Meyer:	160
Rory Shea:	210
Andre Connell:	193
Hendrik de Ridder:	80
Andrew Davies:	1000
John Leuner:	700
Chris Welsh:	700
Gordon Wells:	350
Peet Brits:	353
Hendrik de Ridder:	273
Elizabeth Meyer:	298
Andre Connell:	193
Samuel Scott:	730
Paul Steyn:	573
Francois van Niekerk:	185
Rory Shea:	210
Christiaan Maasdorp:	130
Andrew Davies:	1000
Peet Brits:	573
Christiaan Maasdorp:	392
Rory Shea:	392
Elizabeth Meyer:	365
Chris Welsh:	700
Andre Connell:	193
Samuel Scott:	730
Hendrik de Ridder:	273
Gordon Wells:	350
Paul Steyn:	573
Francois van Niekerk:	185
John Leuner:	700


calculate participation points

In [474]:
game_type = {0.5 : 'i', 1.0 : 'c', 1.5 : 't' , 0.0 : 'n'}
import datetime

for line in data['games'].iterrows():
    white = line[1]['white']
    black = line[1]['black']
    gt = game_type[line[1]['type']]
    newdate = datetime.datetime.strptime(line[1]['date'],'%d/%m/%Y').strftime('%Y%m%d')
    
    if white in players.keys():
        if black in players.keys():                        
            players[white].calc_part(newdate,players[black],gt)
        else:
            players[white].calc_part(newdate,player(black,'',0),gt)
    
    if black in players.keys():
        if white in players.keys():
            players[black].calc_part(newdate,players[white],gt)
        else:
            players[black].calc_part(newdate,player(white,'',0),gt)

for name in players.keys():
    part = players[name].part
    if part > 0:
        print ("%s:\t%s"%(name,part))

Elizabeth Meyer:	1000
Peet Brits:	1000
James Nicolson:	240
Lloyd Rubidge:	740
Andre Connell:	1000
Ben Bredenkamp:	420
Victor Chow:	280
Paul Edwards:	120
Samuel Scott:	980
Welile Gogotshe:	460
Gordon Wells:	360
Chris Visser:	20
Margot Smythe:	300
Brett Wilson:	180
Paul Steyn:	820
Allen Simpson:	80
Antoine Dymond:	520
Andrew Davies:	1000
Jacobus van der Merwe:	140
John Leuner:	1000
Francois van Niekerk:	120
Marc Loon:	820
Michael Yang:	80
Rory Shea:	860
Arnold Wentzel:	400
Chris Welsh:	1000
Bob Gale:	700
Harvey Zhang:	200
Christiaan Maasdorp:	440
Hendrik de Ridder:	240
Kim He-Jin:	180


calculate closing balances

In [475]:
for name in players.keys():
    players[name].calc_close()
        

sort players by closing balances

In [482]:
players_list=[]

for name in players.keys():
    players_list.append(players[name])

players_list.sort(key=lambda player: player.close)
players_list.reverse()

i = 1
for player in players_list:
    print ("%s:\t%s\t%s"%(i,player.close,player.name))
    i += 1

1:	10856	Victor Chow
2:	9520	Samuel Scott
3:	9303	Andrew Davies
4:	8723	Welile Gogotshe
5:	7757	Andre Connell
6:	7447	Chris Welsh
7:	6608	Lloyd Rubidge
8:	6504	John Leuner
9:	4983	Paul Steyn
10:	4946	Ben Gale
11:	4890	Bob Gale
12:	4358	Rory Shea
13:	4306	David Richfield
14:	4199	Paul Edwards
15:	4005	Steve Kroon
16:	3937	Francois van Niekerk
17:	3635	Konrad Scheffler
18:	3450	Dave Gale
19:	2722	James Nicolson
20:	2454	Sipho Mampe
21:	2345	Ben Bredenkamp
22:	2224	Gordon Wells
23:	2217	Marc Loon
24:	2148	Aki Zhou
25:	2122	Max Rabkin
26:	1991	Chris Visser
27:	1773	Peet Brits
28:	1735	Stephen Martindale
29:	1554	Arnold Wentzel
30:	1539	Thabiso Lemeke
31:	1515	Elizabeth Meyer
32:	1310	Tristen Taylor
33:	1158	Rory Beling
34:	1084	Reinhardt Messerschmidt
35:	1063	James Gelant
36:	982	Christiaan Maasdorp
37:	982	Antoine Dymond
38:	794	Florian Breuer
39:	770	Margot Smythe
40:	747	Cheng Lai
41:	741	Peter Charter
42:	738	Sakkie Buys
43:	715	Leander Gaum
44:	672	Jaco Swanepoel
45:	613	Hendrik de R

Example report

In [484]:
players['Gordon Wells'].report()

['---------------------------------------------\n',
 'WAGC points for Gordon Wells                 \n',
 '---------------------------------------------\n',
 '2014 opening balance:                    1768\n',
 'Annual depreciation:                     -354\n',
 "Representative's depreciation:              0\n",
 'Membership points:                        100\n',
 'SA Championship points:                     0\n',
 'Internet Championship points:             350\n',
 'Participation points:                     360\n',
 '---------------------------------------------\n',
 '2014 closing balance:                    2224\n',
 '---------------------------------------------\n',
 '\n',
 '\n',
 '\n',
 '---------------------------------------------\n',
 'Exposition of Internet Championship points   \n',
 '---------------------------------------------\n',
 'Tournament 1                                0\n',
 'Tournament 2                                0\n',
 'Tournament 3                             